# Libraries used

In [ ]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # split data for testing 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle # to save the model in a file
import os

# If not found, Create and Train the Random Forest Classifier with the data gathered in asl_data.csv

In [ ]:
MODEL_PATH = 'asl_model.pkl'

if not os.path.exists(MODEL_PATH):
    print("Model file not found. Training a new one.")

    # Load dataset
    data = pd.read_csv('asl_dataset/asl_data.csv')

    X = data.iloc[:, 1:] # All rows, all columns except the first one
    y = data.iloc[:, 0]  # All rows, only the first column (The labels)

    # Training set is 80%, test set is 20%
    # stratify=y ensures a balanced mix of letters in both sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Create the Random Forest Classifier model
    # 100 decision trees to vote on the answer
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the model
    # look at X_train and y_train to learn patterns
    model.fit(X_train, y_train)

    # Test the model
    # Predict the letters for the X_test data
    y_pred = model.predict(X_test)

    # Calculate the accuracy by comparing predicted letters with actual letters
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model trained with accuracy: {accuracy*100:.2f}%")

    # Save the model to the file
    with open(MODEL_PATH, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"Model saved to {MODEL_PATH}")

# Use the model to Predict the Letter live

In [ ]:
with open(MODEL_PATH, 'rb') as f:
    model = pickle.load(f)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

webCam = cv.VideoCapture(0)

while webCam.isOpened():

    success, frame = webCam.read()

    if not success:
        break

    frame = cv.flip(frame, 1)
    rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:

        hand_landmarks = results.multi_hand_landmarks[0]

        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        wrist_coords = hand_landmarks.landmark[0]
        landmarks_data = []

        for lm in hand_landmarks.landmark:
            normalized_x = lm.x - wrist_coords.x
            normalized_y = lm.y - wrist_coords.y
            normalized_z = lm.z - wrist_coords.z

            landmarks_data.extend([normalized_x, normalized_y, normalized_z])
        
        #reshaping to a 2d array with 1 row and multiple columns for the model input
        landmarks_data = np.array(landmarks_data).reshape(1, -1)

        # Prediction using the trained model
        prediction = model.predict(landmarks_data)
        predicted_letter = prediction[0]

        confidence = model.predict_proba(landmarks_data).max()

        cv.putText(frame, f'Letter: {predicted_letter} ({confidence*100:.2f}%)', (10, 70), cv.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv.LINE_AA)

    cv.imshow('ASL Recognizer', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

webCam.release()
cv.destroyAllWindows()
cv.waitKey(500)